In [214]:
from sklearn import model_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import *
%matplotlib inline 
%config InlineBackend.figure_format="retina" 

In [215]:
# 数据处理    
def ReadData(pathname):
    data=pd.read_csv(pathname)
    data.loc[data['好瓜']=='是', '好瓜'] = 1
    data.loc[data['好瓜']=='否', '好瓜'] = 0
    data = data.iloc[:,1:]
    
    attribute=data.columns[:-1]
    label=data['好瓜']
    
    data=disperData(data,label)

    arr=np.array(data)
     
    return arr,attribute

# 集离散化
def disperData(df,classLabel):
    newData=df
    feat=[["浅白","青绿","乌黑"],["蜷缩","稍蜷","硬挺"],["清脆","浊响","沉闷"],["模糊","稍糊","清晰"],["凹陷","稍凹","平坦"],["硬滑","软粘"]]
    
    feature=pd.DataFrame(feat)
    # 转置
    feature = pd.DataFrame(feature.values.T)
    feature.columns=["色泽","根蒂","敲声","纹理","脐部","触感"]
    
    for index in df.columns:
        if index!="含糖率" and index!="密度" and index!="好瓜":
            for row in df.index:
                # 获取所在行索引值
                value=df.loc[row,index]
                df.loc[row,index]=feature.loc[feature[index]==value].index+1
    
    return df

In [216]:
def sigmoid(Z):
    return 1.0/(1+np.exp(-Z))

In [241]:
# 标准BP算法
# hideNum为隐层神经元个数
def BP(X,Y,hideNum=5,eta=0.01,epoch=10000):

    # 权值及偏置初始化
    V = np.random.rand(X.shape[1],hideNum)
    V_b = np.random.rand(1,hideNum)
    W = np.random.rand(hideNum,Y.shape[1])
    W_b = np.random.rand(1,Y.shape[1])

    trainNum=0
    totloss=0
    
    while trainNum<epoch:
        # 标准BP每次处理一个样本
        for k in range(X.shape[0]):
            B_h=sigmoid(X[k,:].dot(V)-V_b) # 输入层-隐层 注意是减去阈值
            Y_=sigmoid(B_h.dot(W)-W_b)     # 隐层-输出层 注意是减去阈值
            loss=sum((Y[k]-Y_)**2)*0.5      # 算均方误差
            
            
            # 计算梯度并更新参数
            g=Y_*(1-Y_)*(Y[k]-Y_)
            e=B_h*(1-B_h)*g.dot(W.T)
            
            # 参数更新
            W+=eta*B_h.T.dot(g)
            W_b-=eta*g
            V+=eta*X[k].reshape(1,X[k].size).T.dot(e)
            V_b-=eta*e
            trainNum+=1
    
    
    print("标准BP")
    print("总训练次数：",trainNum)
    print("最终损失：",loss)
    #print("V：",V)
    #print("V_b：",V_b)
    #print("W：",W)
    #print("W_b：",W_b)

        

In [242]:
# 累积BP算法
def BPAcc(X,Y,hideNum=5,eta=0.01,epoch=10000):
    
    # 权值及偏置初始化
    V = np.random.rand(X.shape[1],hideNum)
    V_b = np.random.rand(1,hideNum)
    W = np.random.rand(hideNum,Y.shape[1])
    W_b = np.random.rand(1,Y.shape[1])
    
    trainNum=0
    
    while trainNum<epoch:
        # 累积BP直接处理所有样本
        B_h=sigmoid(X.dot(V)-V_b)   # 输入层-隐层 注意是减去阈值
        Y_=sigmoid(B_h.dot(W)-W_b)  # 隐层-输出层 注意是减去阈值
        loss=0.5*sum((Y-Y_)**2)/X.shape[0]     # 算均方误差
        
        # 计算梯度并更新参数
        g=Y_*(1-Y_)*(Y-Y_)
        e=B_h*(1-B_h)*g.dot(W.T)
            
        # 参数更新
        W+=eta*B_h.T.dot(g)
        W_b-=eta*g.sum(axis=0)
        V+=eta*X.T.dot(e)
        V_b-=eta*e.sum(axis=0)
        trainNum+=1
        
    print("累积BP")
    print("总训练次数：",trainNum)
    print("最终损失：",loss)
    #print("V：",V)
    #print("V_b：",V_b)
    #print("W：",W)
    #print("W_b：",W_b)


In [243]:
if __name__ == '__main__':
    data,attribute=ReadData('./watermelon_3.csv')
    
    X=data[:,:-1]
    y=data[:,-1]
    y=y.reshape(y.shape[0],1)
    BP(X,y)
    BPAcc(X,y)

标准BP
总训练次数： 10013
最终损失： 0.10798687961100382
累积BP
总训练次数： 10000
最终损失： 0.02651380560393347
